# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Загрузка библиотек 

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve

import warnings
warnings.filterwarnings('ignore')

RND = 12345

### Загрузка данных 

In [2]:
try:
    toxic_comments = pd.read_csv('toxic_comments.csv')    
except:
    toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')

### Обработка данных 

In [3]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
toxic_comments.head(3)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0


In [5]:
toxic_comments['toxic'].value_counts(0.1)*100

0    89.838787
1    10.161213
Name: toxic, dtype: float64

- Пропуски отсутсвуют
- Целевой признак `toxic`
- 89.84% текстов не токсичны, 10.16% токсичны

### Обработка текста

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vladkondratov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/vladkondratov/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vladkondratov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()

In [10]:
def lemmatize_text(text):
    text = text.lower()
    lemm_text = " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(text)])
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text)
    return ''.join(cleared_text)

In [11]:
toxic_comments_lem = toxic_comments.copy()
toxic_comments_lem.drop(['text'], axis=1, inplace=True)

In [12]:
%%time

toxic_comments_lem['text'] = toxic_comments['text'].apply(lemmatize_text)

CPU times: user 1min 38s, sys: 233 ms, total: 1min 38s
Wall time: 1min 38s


## Обучение

### Выборки

In [14]:
features = toxic_comments_lem.drop('toxic', axis=1)
target = toxic_comments_lem['toxic']

In [15]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.2, random_state=RND)

In [16]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [17]:
features_train = count_tf_idf.fit_transform(features_train['text'])
features_test = count_tf_idf.transform(features_test['text'])

In [18]:
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

(127433, 140996)
(127433,)
(31859, 140996)
(31859,)


### LogisticRegression

In [19]:
%%time

classificator = LogisticRegression()
hyperparams = [{'solver':['newton-cg', 'lbfgs', 'liblinear'],
                'C':[0.1, 1, 10],
                'class_weight':['balanced', None]}]

clf = GridSearchCV(classificator, hyperparams, scoring='f1',cv=3)
clf.fit(features_train, target_train)

lr_best_params = clf.best_params_

CPU times: user 4min 7s, sys: 13 s, total: 4min 20s
Wall time: 1min 23s


In [20]:
print(lr_best_params)
print()
print("F1 score: {:.3f}".format(clf.best_score_))

{'C': 10, 'class_weight': None, 'solver': 'newton-cg'}

F1 score: 0.759


### CatBoostClassifier

In [21]:
%%time

classificator = CatBoostClassifier() 
hyperparams = [{'learning_rate':[0.1, 0.2, 0.3],
                'random_state':[RND],
                'iterations' : [100],
                'verbose':[False]}]

cb = GridSearchCV(classificator, hyperparams, scoring='f1')
cb.fit(features_train, target_train)

cb_best_params = cb.best_params_

CPU times: user 1h 42min 45s, sys: 1min 39s, total: 1h 44min 24s
Wall time: 18min 42s


In [22]:
print(cb_best_params)
print()
print("F1 score: {:.3f}".format(cb.best_score_))

{'iterations': 100, 'learning_rate': 0.3, 'random_state': 12345, 'verbose': False}

F1 score: 0.713


In [23]:
columns = ['Модель','F1 score']
logistic_model = ['Логистическая регрессия', "{:.3f}".format(clf.best_score_)]
cb_model = ['CatBoostClassifier', "{:.3f}".format(cb.best_score_)]
table = pd.DataFrame([logistic_model, cb_model], columns = columns)

display(table)

,Модель,F1 score
0,Логистическая регрессия,0.759
1,CatBoostClassifier,0.713


In [24]:
lr_model = LogisticRegression()
lr_model.set_params(**lr_best_params)

LogisticRegression(C=10, solver='newton-cg')

In [25]:
lr_model.fit(features_train, target_train)
lr_pred = lr_model.predict(features_test)

In [26]:
test_score = f1_score(target_test, lr_pred)

In [27]:
print("F1 score: {:.3f}".format(test_score))

F1 score: 0.779


## Вывод

- Данные подготовлены.
- Сформированы обучающая и тестовая выборка.
- Обучены модели LogisticRegression и CatBoostClassifier.
- Подобраны наилучшие параметры для моделей.
- Выбрана модель с наилучшим f1 score для проверки на тестовой выборке.
- f1 score лучшей модели на тестовой выборке 0.779